In [ ]:
import numpy as np
from PIL import Image

from keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.applications import resnet50

from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

print("Loading CIFAR10 images ...")
(Xtrain, Ytrain), (Xtest, Ytest) = cifar10.load_data()

print('\tOriginal training set shape: ', Xtrain.shape)
print('\tOriginal testing set shape: ', Xtest.shape)

Loading CIFAR10 images ...
	Original training set shape:  (50000, 32, 32, 3)
	Original testing set shape:  (10000, 32, 32, 3)


In [ ]:
def lowSampleDataset(X, Y):
    perm = np.random.permutation(X.shape[0])
    X = X[perm[0 : (int)(X.shape[0] * (5/100))]]
    Y = Y[perm[0 : (int)(Y.shape[0] * (5/100))]]
    return X, Y

Xtrain, Ytrain = lowSampleDataset(Xtrain, Ytrain)
Xtest, Ytest = lowSampleDataset(Xtest, Ytest)

print('\tOriginal training set shape: ', Xtrain.shape)
print('\tOriginal testing set shape: ', Xtest.shape)

	Original training set shape:  (2500, 32, 32, 3)
	Original testing set shape:  (500, 32, 32, 3)


In [ ]:
X = []
for i in range(0, Xtrain.shape[0]):
    X.append(np.array(Image.fromarray(Xtrain[i]).resize(size=(224,224))))
Xtrain = np.array(X)

X = []
for i in range(0, Xtest.shape[0]):
    X.append(np.array(Image.fromarray(Xtest[i]).resize(size=(224,224))))
Xtest = np.array(X)

print('\tTraining set shape: ', Xtrain.shape)
print('\tTesting set shape: ', Xtest.shape)

	Training set shape:  (2500, 224, 224, 3)
	Testing set shape:  (500, 224, 224, 3)


In [ ]:
print("Loading the ResNet50-ImageNet model ...")
model = resnet50.ResNet50(include_top=True, weights='imagenet', input_shape=(224, 224, 3), classes=1000)
model = Model(inputs=model.input, outputs=model.get_layer(index=-2).output)
#model.summary()

prediction = np.array(model.predict(Xtrain))
Xtrain = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

prediction = np.array(model.predict(Xtest))
Xtest = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

print('\tFeatures training shape: ', Xtrain.shape)
print('\tFeatures testing shape: ', Xtest.shape)

In [ ]:
print("Classification with KNN ...")
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(Xtrain, np.ravel(Ytrain, order='C'))
result = knn.predict(Xtest)

acc = accuracy_score(result, Ytest)
print("\tAccuracy Linear KNN: %0.4f" % acc)

Classification with KNN ...
	Accuracy Linear KNN: 0.6220


# Exercícios:
1. Considerando o dataset CIFAR10, faça extração de características com a antepenúltima camada.  
* Indique a quantidade de atributos obtidos nesta camada.
* Utilize o espaço de características com o classificador KNN com 3 vizinhos. Compare o resultado obtido com a versão de exemplo. Qual a melhor performance?
* Repita este mesmo experimento executando PCA como etapa complementar entre a extração e classificação. Compare novamente os resultados.

In [ ]:
# Para extração de caracteristicas na penultima camada, precisamos que a saída da rede seja no indice -3

print("Loading the ResNet50-ImageNet model ...")
model = resnet50.ResNet50(include_top=True, weights='imagenet', input_shape=(224, 224, 3), classes=1000)
model = Model(inputs=model.input, outputs=model.get_layer(index=-3).output)

Loading the ResNet50-ImageNet model ...


In [ ]:
print("Loading CIFAR10 images ...")
(Xtrain, Ytrain), (Xtest, Ytest) = cifar10.load_data()

print('\tOriginal training set shape: ', Xtrain.shape)
print('\tOriginal testing set shape: ', Xtest.shape)

Xtrain, Ytrain = lowSampleDataset(Xtrain, Ytrain)
Xtest, Ytest = lowSampleDataset(Xtest, Ytest)

print('\tSampled training set shape: ', Xtrain.shape)
print('\tSampled testing set shape: ', Xtest.shape)

X = []
for i in range(0, Xtrain.shape[0]):
    X.append(np.array(Image.fromarray(Xtrain[i]).resize(size=(224,224))))
Xtrain = np.array(X)

X = []
for i in range(0, Xtest.shape[0]):
    X.append(np.array(Image.fromarray(Xtest[i]).resize(size=(224,224))))
Xtest = np.array(X)

print('\tFinal Training set shape: ', Xtrain.shape)
print('\tFinal Testing set shape: ', Xtest.shape)

Loading CIFAR10 images ...
	Original training set shape:  (50000, 32, 32, 3)
	Original testing set shape:  (10000, 32, 32, 3)
	Sampled training set shape:  (2500, 32, 32, 3)
	Sampled testing set shape:  (500, 32, 32, 3)
	Final Training set shape:  (2500, 224, 224, 3)
	Final Testing set shape:  (500, 224, 224, 3)


In [ ]:
prediction = np.array(model.predict(Xtrain))
Xtrain = np.reshape(prediction, (prediction.shape[0], 100352))

prediction = np.array(model.predict(Xtest))
Xtest = np.reshape(prediction, (prediction.shape[0], 100352))

print('\tFeatures training shape: ', Xtrain.shape)
print('\tFeatures testing shape: ', Xtest.shape)

	Features training shape:  (2500, 100352)
	Features testing shape:  (500, 100352)


In [ ]:
print("Classification with KNN ...")
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(Xtrain, np.ravel(Ytrain, order='C'))
result = knn.predict(Xtest)

acc = accuracy_score(result, Ytest)
print("\tAccuracy Linear KNN: %0.4f" % acc)

Classification with KNN ...
	Accuracy Linear KNN: 0.4680


A melhor performance foi obtida na versão de exemplo

In [ ]:
# Aplicando PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=.95, svd_solver='full')
Xtrain_pca = pca.fit_transform(Xtrain)
Xtest_pca = pca.transform(Xtest)

print("Classification with KNN ...")
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(Xtrain_pca, np.ravel(Ytrain, order='C'))
result = knn.predict(Xtest_pca)

acc = accuracy_score(result, Ytest)
print("\tAccuracy Linear KNN: %0.4f" % acc)

Classification with KNN ...
	Accuracy Linear KNN: 0.4180


Os resultados com PCA foram ainda piores que os outros dois exemplos.